In [1]:
from sparsel0 import *
from scipy.io import loadmat
import pandas as pd
import timeit

In [2]:
    numerator = 0
    denominator = 0
    numerator1 = 0
    denominator1 = 0
    SNR = []
    RSNR = []
    time = []
    num = 100

    for k in range(num):
        print(k)
        A = loadmat('Experiment1/50/A.mat')['A']
        b = loadmat(f'Experiment1/50/b{k + 1}.mat')['b'].T[0]
        x_true = loadmat(f'Experiment1/50/x{k + 1}.mat')['x_true']
        x_true = x_true.toarray().squeeze()

        SNR.append(10 * np.log10(np.linalg.norm(A @ x_true) ** 2 / np.linalg.norm(b - A @ x_true) ** 2))
        numerator += np.linalg.norm(A @ x_true) ** 2
        denominator += np.linalg.norm(b - A @ x_true) ** 2

        numerator1 += np.linalg.norm(x_true) ** 2

        lambda_ = 2

        # You need to implement or import the sparse_l0 function
        out = sparse_l0(A, b, lambda_)
        start_time = timeit.default_timer()
        out.step()
        end_time = timeit.default_timer()
        elapsed_time = end_time - start_time

        indt = np.where(x_true > 0)[0]
        indx = np.where(out.x > 0)[0]
        # common_elements = np.intersect1d(indt, indx)
        # print(common_elements)

        RSNR.append(10 * np.log10(np.linalg.norm(x_true) ** 2 / np.linalg.norm(out.x - x_true) ** 2))
        time.append(elapsed_time)
        denominator1 += np.linalg.norm(out.x - x_true) ** 2

0
Optimization begins! The initialization step starts. 
Iteration: 0, objective value = 7.096295676048335, gradient norm = 89.10717279349196.
The initialization step finishes!
The current iteration is 40, the objective value is 0.0024754664728640404.
The L0BPG step starts.
Iteration: 0, objective value = 14.004108320964148, nonzero number = 7.
Iteration: 10, objective value = 14.003375230015504, nonzero number = 7.
Iteration: 20, objective value = 14.00317373500113, nonzero number = 7.
Iteration: 30, objective value = 14.003017035559175, nonzero number = 7.
Iteration: 40, objective value = 14.002890851309278, nonzero number = 7.
Iteration: 50, objective value = 14.002789411505546, nonzero number = 7.
Iteration: 60, objective value = 14.002708178933217, nonzero number = 7.
Iteration: 70, objective value = 14.002643368605169, nonzero number = 7.
Iteration: 80, objective value = 14.002591832060165, nonzero number = 7.
Iteration: 90, objective value = 14.002550971383767, nonzero number = 7

In [3]:
    SNR_mean = 10 * np.log10(numerator / denominator)
    RSNR_mean = 10 * np.log10(numerator1 / denominator1)

    rowNames = [str(i + 1) for i in range(num)] + ['mean']
    result = np.column_stack((SNR, [SNR_mean] * num, RSNR, [RSNR_mean] * num, time, [2] * num))
    result = np.vstack((result, np.mean(result, axis=0)))
    colNames = ['SNR', 'SNR_mean', 'RSNR', 'RSNR_mean', 'time', 'lambda']
    result = pd.DataFrame(result, columns=colNames, index=rowNames)
    print(result)

       SNR  SNR_mean       RSNR  RSNR_mean      time  lambda
1     50.0      50.0  40.447909  45.381827  0.029174     2.0
2     50.0      50.0  60.988350  45.381827  0.060344     2.0
3     50.0      50.0  59.122796  45.381827  0.086063     2.0
4     50.0      50.0  60.013100  45.381827  0.049457     2.0
5     50.0      50.0  67.303235  45.381827  0.063289     2.0
...    ...       ...        ...        ...       ...     ...
97    50.0      50.0  65.564400  45.381827  0.111965     2.0
98    50.0      50.0  64.423114  45.381827  0.136428     2.0
99    50.0      50.0  66.041601  45.381827  0.050921     2.0
100   50.0      50.0  62.996741  45.381827  0.037228     2.0
mean  50.0      50.0  57.755781  45.381827  0.054145     2.0

[101 rows x 6 columns]
